In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import sklearn
from sklearn.utils import resample
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

In [ ]:
path = r'path\to\array.csv'
data = np.loadtxt(path, delimiter=",")

In [ ]:
# Ignore reference value
X = data[:, 0:100]

def get_ab(avalanche_number, k, degrees):
  # Place the ranges in a variable and preprocess
  x = np.arange(100)
  y = X[avalanche_number]
  x = x.reshape(-1, 1)

  # Change the order here. degree is same as M
  poly = PolynomialFeatures(degree=7)

  # Fit a Polynomial Curve
  X_poly = poly.fit_transform(x)
  poly.fit(X_poly, y)
  linreg = LinearRegression()
  linreg.fit(X_poly, y)
  y_pred = linreg.predict(X_poly)

  # Calculate the 10 degree threshold and gradient of y_pred
  x = np.arange(100)
  thd = np.tan(np.radians(-degrees))
  f = np.repeat(thd, 100, axis=None)
  g = np.gradient(y_pred)
  idx = np.argwhere(np.diff(np.sign(f - g))).flatten()

  # Choose first beta value after k
  beta_values = np.append(x[idx], 100)
  min_beta = min(filter(lambda i: i > k, beta_values))

  # Calculate beta angle
  tan = y[0]/min_beta
  beta = np.arctan(tan)
  beta = np.degrees(beta)

  # Calculate alpha angle
  tan = y[0]/100
  alpha = np.arctan(tan)
  alpha = np.degrees(alpha)

  if beta == alpha:
    beta = 0

  return beta, alpha

In [ ]:
# Loop through all avalanches and calculate alpha and beta angles for each avalanche
res_array = np.empty((0, 2))
rg = range(0, 18658, 1)
k = 50 # 5 or 50
beta_degrees = 10 # 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 20, 25 or 30

for i in rg:
  try:
    result = get_ab(i, k, beta_degrees)
    res_array = np.append(res_array, [result], axis=0)
  except Exception:
    pass